In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np
from datetime import datetime 
from datetime import timedelta
import redis
import os
import torch
import threading
import time

In [3]:
## 변수 모음
stock = '031440'

today_date_str = datetime.now().strftime('%Y%m%d')
base_path = '/home/hyunsu3408/dl-dev/rl-dev/rltrader-master/output/'

#C:\big16\stock\rltrader-master\src

In [80]:
def chan_redis(send_channel_name, info): # 수익률 보내기
    
    c = redis.Redis(host='34.64.63.106', port=6379, db=0)
    c.publish(send_channel_name, info)

In [ ]:
def insert_pv(stock, date, pv):
    conn = pymysql.connect(host='34.64.197.181', user='SUJIN', password='SUJIN',
                        db='shin_db', charset='utf8')
    curs = conn.cursor()
    sql = '''INSERT IGNORE INTO PV_{} VALUES (%s, %s)'''.format(stock)
    
    val = (str(date), int(pv))
    curs.execute(sql, val)
    conn.commit()
    conn.close()

In [ ]:
def send_trading(send_channel_name,trading_unit):
    r = redis.Redis(host='34.64.37.236', port=6379, db=0)
    r.publish(send_channel_name, trading_unit)

In [81]:
## action 보내는 행동
def send_redis(pred_path, send_channel_name, acc, stock):
    df = pd.read_json(pred_path)
    bsy=df[1].iloc[-1]
    prob= df[2].iloc[-1]
    
    if bsy==0: # 매수
        trading_unit= int(((prob-0.50)/0.01)+1)
        trading_unit= max(trading_unit,1)
        trading_unit= min(trading_unit,100) # max 100
        print('********************************')
        print('{} 종목 {}개 매수합니다'.format(stock,int(trading_unit)))
        acc = acc+1
        action='2'
        
    elif bsy==1: # 매도
        trading_unit= int(((prob-0.50)/0.01)+1)
        trading_unit= max(trading_unit,1)
        trading_unit= min(trading_unit,100) # max acc or 100
        print('********************************')
        print('{} 종목 {}개 매도합니다'.format(stock,int(trading_unit)))
        acc = acc-1
        action='1'
        
    elif bsy==2: # 관망
        print('********************************')
        print('{} 종목 홀드합니다'.format(stock))
        trading_unit=0
        action='0'
        acc = acc
        
    c = redis.Redis(host='34.64.63.106', port=6379, db=0) # redis로 보냄니다
    if action=='0':
        c.publish(send_channel_name,'관망')
    if action=='1':
        c.publish(send_channel_name,'매도')
    if action=='2':
        c.publish(send_channel_name,'매수')
        
    r = redis.Redis(host='34.64.37.236', port=6379, db=0) # 다른 redis로 보냅니다
    r.publish(send_channel_name,action)

    return acc,int(trading_unit)

In [82]:
## 잔고 등 정보 받는 행동
def get_redis(get_channel_name):
    r = redis.Redis(host='34.64.37.236', port=6379, db=0)
    s = r.pubsub()
    s.subscribe(get_channel_name)
    i = 0
    while True :
        res = s.get_message()
        for message in s.listen():
            data = message.get("data")
            if isinstance(data, bytes):
                i=i+1
                break
        if i==1:
            break
    data = str(data,'utf-8')
    print(data)
    return data

In [83]:
## predict 
def predict_command(stock,balance,date1,date2, pred_path):

    pred_command = "python /home/hyunsu3408/dl-dev/rl-dev/rltrader-master/src/main.py --mode predict --ver v3 --name {} --stock_code {} --rl_method a3c --net cnn --balance {} --start_date {} --end_date {}".format(stock, stock, balance, date1, date2)
    os.system(pred_command)

    temp_time = datetime.strptime(date2, '%Y%m%d%H%M')
    next_time = (temp_time+timedelta(minutes=1)).strftime('%Y%m%d%H%M')
    
    print('predict 완료')
    
    df = pd.read_json(pred_path)
    bsy=df[1].iloc[-1] 
    return next_time, bsy

In [84]:
## test 
def test_command(stock,balance,date1,date2):
    pred_command = "python /home/hyunsu3408/dl-dev/rl-dev/rltrader-master/src/main.py --mode test --ver v3 --name {} --stock_code {} --rl_method a3c --net cnn --balance {} --start_date {} --end_date {}".format(stock, stock, balance, date1, date2)
    os.system(pred_command)

    temp_time = datetime.strptime(date2, '%Y%m%d%H%M')
    next_time = (temp_time+timedelta(minutes=1)).strftime('%Y%m%d%H%M')
    
    print('test 완료')
    return next_time

In [85]:
## update 강화학습 
def update_command(stock,after_balance,date1,date2):
    
    upd_command = "python /home/hyunsu3408/dl-dev/rl-dev/rltrader-master/src/main.py --mode update --ver v3 --name {} --stock_code {} --rl_method a3c --net cnn --balance {} --start_date {} --end_date {}".format(stock, stock, after_balance, date1, date2)
    os.system(upd_command)
    
    temp_time = datetime.strptime(date2, '%Y%m%d%H%M')
    next_time = (temp_time+timedelta(minutes=1)).strftime('%Y%m%d%H%M')
    print('update 완료')
    return next_time

In [86]:
## 종가 and csv속 첫시간, csv속 마지막시간 가져오기
def read_close_date_stock(stock,date=0):

    df = pd.read_csv('/home/hyunsu3408/dl-dev/rl-dev/rltrader-master/data/v3/{}_신세계푸드.csv'.format(stock), index_col=0)
    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)
    if date==0: # 종가 가져오고 싶으면 시간을 넣어야함
        close=0
    else:
        close = df[df['date']==int(date)]['close'].iloc[0]

    return str(df['date'][0]), close, str(df['date'][df.shape[0]-1]), str(df['date'][df.shape[0]-6])

In [87]:
def runrunrun(stock, acc):
    pred_file='predict_{}_a3c_cnn'.format(stock)+'/pred_{}.json'.format(stock)
    pred_path = base_path+pred_file
    
    print(stock)
    print("[Redis] balance을 기다리는 중입니다")
    time.sleep(2)
    initial_balance = get_redis(stock+'balance_channel')
    print('[Redis] 대신 api에서 가져온 현재 잔고는 {}입니다'.format(initial_balance))
    now_time11 = (datetime.now()-timedelta(minutes=0)).strftime('%Y%m%d%H%M')
    BBB = datetime.now().strftime('%H:%M:%S')
    print('현재시각은 {}입니다 20초에 predict 시작합니다'.format(BBB))
    ########### 하루의 첫 거래 # 데이터 안들어올거를 대비 9시10분을 첫 학습
    while True:

        now_time = datetime.now().strftime('%S')
        if now_time=='20': #############
            if int(now_time11)>int(today_date_str+'0910'):
                print('[시스템] 첫거래 predict합니다')
                first_date, _, last_date, sixth = read_close_date_stock(stock=stock)
                next_time, act = predict_command(stock,initial_balance,first_date,last_date, pred_path)
                acc,trading_unit = send_redis(pred_path, stock+'action_channel', acc, stock)
                time.sleep(7)
                send_trading(stock+'trading_channel',trading_unit)
                print("[Redis] 잔고를 기다리는 중입니다")
                after_balance = get_redis(stock+'balance_channel')
                print('[Redis] 첫거래후 잔고는 {}입니다'.format(after_balance))

                break

    BBB = datetime.now().strftime('%H:%M:%S')
    print('===============================')
    print('현재시각은 {}입니다'.format(BBB))
    # 첫거래후 수익률 계산하는 부분
    print("[Redis] 체결종가를 기다리는 중입니다")
    real_close = get_redis(stock+'real_close') # 체결된 종가
    print('첫거래후 대신 api에서 체결종가는 {}입니다'.format(real_close))
    print('20초에 predict 시작합니다')
    _, now_close, _, _ = read_close_date_stock(stock, last_date) # 기록된 종가
    if int(real_close)>0:
        performance_print = (now_close-int(real_close))/int(real_close)*100
        performance_rate = abs(performance_print)
    else:
        performance_print=0
        performance_rate=0

    now_ = datetime.now().strftime('%Y%m%d%H%M')

    while True:
#         print("=====================================")
        now_time = datetime.now().strftime('%H%M')
        if int(now_time) > 1521: #15시20분이면 종료
            print("현재 시각 {} 정상으로 종료합니다".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
            break

        else:
            
            # 디비속의 최신 데이터 가져오기, 현재 시간과 관계 없음
            _, _, last_date, sixth = read_close_date_stock(stock)
            last_tmp = datetime.strptime(last_date, '%Y%m%d%H%M')
            last_5 = (last_tmp-timedelta(minutes=6)).strftime('%Y%m%d%H%M') # before

            now_min1 = datetime.now().strftime('%M')
            now_sec = datetime.now().strftime('%S')

            if now_sec =='20':
                print("=====================================")
                BBB = datetime.now().strftime('%H:%M:%S')
                print('현재시각은 {}입니다'.format(BBB))
                if performance_rate >=0.3 : # 임계 지연 보상 신세계푸드
                    if real_close==0:
                        continue
                    print('수익률 {}가 나왔습니다, 모델을 update합니다'.format(performance_print))

                    if int(last_date)-int(next_time)>5:
                        update_command(stock, after_balance, next_time, last_date) # 업데이트
                    else:
                        update_command(stock, after_balance, sixth, last_date) # 업데이트
                    
                    next_time, act = predict_command(stock, after_balance, sixth, last_date, pred_path) 
                    # 50epoch update가 1분내로 된다는 가정하에 predict
                    
                    time.sleep(15)
                    acc,trading_unit = send_redis(pred_path, stock+'action_channel', acc, stock)
                    chan_time =  datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    chan_redis(stock+"timepv",chan_time)
                    time.sleep(7)
                    send_trading(stock+'trading_channel',trading_unit)
                    print('[Redis] 대신api에 action을 보냈습니다')
                    print("[Redis] balance을 기다리는 중입니다")
                    time.sleep(3)
                    after_balance = get_redis(stock+'balance_channel')
                    print("대신api에서 받은 잔고는 {}입니다.".format(after_balance))

                    _, now_close, _, _ = read_close_date_stock(stock,last_date)
                    print('마지막 종가는 {}입니다'.format(now_close))
                    print("[Redis] real_close을 기다리는 중입니다")
                    real_close = get_redis(stock+'real_close')

                    print('실제 체결 단가는 {}입니다'.format(real_close))
                    if int(real_close)>0:
                        performance_print = (now_close - int(real_close))/int(real_close)*100
                        performance_rate = abs(performance_print)
                    else:
                        performance_print=0
                        performance_rate=0
                    
                    PV = int(after_balance) + int(real_close)*int(acc)
                    
                    chan_redis(stock+"pv", PV)
                    insert_pv(stock, chan_time, PV)

                    BBB = datetime.now().strftime('%H:%M:%S')
                    print('현재시각은 {}입니다 20초에 predict 시작합니다'.format(BBB))

                    
                else:
                    print('수익률 {}가 나왔습니다, predict합니다'.format(performance_print))
                    
                    next_time, act = predict_command(stock, after_balance, sixth, last_date, pred_path)
                    print(act)
                    
                    time.sleep(15)
                    print('[Redis] 대신api에 action을 보냈습니다')
                    acc,trading_unit = send_redis(pred_path, stock+'action_channel',acc, stock)
                    chan_time =  datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    chan_redis(stock+"timepv",chan_time)
                    time.sleep(7)
                    send_trading(stock+'trading_channel',trading_unit)
                    print("[Redis] balance을 기다리는 중입니다")
                    time.sleep(3)
                    after_balance = get_redis(stock+'balance_channel')
                    print("[Redis] 대신api에서 받은 잔고는 {}입니다.".format(after_balance))

                    _, now_close, _, _ = read_close_date_stock(stock,last_date)
                    print('마지막 종가는 {}입니다'.format(now_close))
                    print("[Redis] real_close을 기다리는 중입니다")
                    real_close = get_redis(stock+'real_close')
                    print('실제 체결 단가는 {}입니다'.format(real_close))
                    if int(real_close)>0:
                        performance_print = (now_close - int(real_close))/int(real_close)*100
                        performance_rate = abs(performance_print)
                    else:
                        performance_print=0
                        performance_rate=0

                    PV = int(after_balance) + int(real_close)*int(acc)
                    chan_redis(stock+"pv", PV)
                    insert_pv(stock, chan_time, PV)

                    BBB = datetime.now().strftime('%H:%M:%S')

                    print('현재시각은 {}입니다 20초에 predict 시작합니다'.format(BBB))
                        

                while True: # 최소 1분에 한번만 돌아가게끔
                    now_min2 = datetime.now().strftime('%M')
                    if now_min1!=now_min2:
                        break


In [ ]:
acc = 0
runrunrun(stock,acc)